<a href="https://colab.research.google.com/github/Adyypower/LLMs-Model/blob/main/Chat_With_PDF_Using_Langchain_And_Astradb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q langchain cassandra-driver astrapy pypdf sentence-transformers "transformers>=4.38" "torch>=2.1"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 333.5/333.5 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 69.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.1/331.1 kB 18.3 MB/s eta 0:00:00


In [ ]:
import os
from google.colab import userdata

In [ ]:
try:
    # This will attempt to load the credentials from Colab Secrets.
    ASTRA_DB_API_ENDPOINT = userdata.get('ASTRA_DB_API_ENDPOINT')
    ASTRA_DB_APPLICATION_TOKEN = userdata.get('ASTRA_DB_APPLICATION_TOKEN')
    print("✅ Astra DB credentials loaded successfully from Colab Secrets!")
except (KeyError, TypeError):
    print("⚠️ Secure credentials not found in Colab Secrets.")

✅ Astra DB credentials loaded successfully from Colab Secrets!


In [ ]:
import requests
pdf_url = "https://arxiv.org/pdf/1706.03762.pdf"
pdf_path = "attention_is_all_you_need.pdf"

try:
    response = requests.get(pdf_url)
    response.raise_for_status() # Raise an exception for bad status codes
    with open(pdf_path, 'wb') as f:
        f.write(response.content)
    print(f"✅ Successfully downloaded '{pdf_path}'")
except requests.exceptions.RequestException as e:
    print(f"❌ Error downloading PDF: {e}")

✅ Successfully downloaded 'attention_is_all_you_need.pdf'


In [ ]:
!pip install langchain-community

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Cassandra
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline


In [ ]:
loader = PyPDFLoader(pdf_path)
documents = loader.load()
print(f"Loader{len(documents)} pages from the research paper")

Loader15 pages from the research paper


In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000,chunk_overlap = 100)
docs = text_splitter.split_documents(documents)
print(f"{len(docs)} chunks of text")

49 chunks of text


In [ ]:
my_embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
model_id = "google/flan-t5-base"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

pipe = pipeline('text2text-generation',model = model,tokenizer = tokenizer,max_new_tokens = 256)
llm = HuggingFacePipeline(pipeline =  pipe)

/tmp/ipython-input-26224411.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  my_embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Device set to use cpu
/tmp/ipython-input-26224411.py:8: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline =  pipe)


In [ ]:
!pip install -q "langchain-astradb>=0.3.2"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.3 MB/s eta 0:00:00


In [ ]:
from langchain_astradb import AstraDBVectorStore


In [ ]:
collection_name = "attention_paper_vectors"
vector_store = AstraDBVectorStore.from_documents(
    documents=docs,
    embedding=my_embeddings,
    collection_name=collection_name,
    token=ASTRA_DB_APPLICATION_TOKEN,
    api_endpoint=ASTRA_DB_API_ENDPOINT,
)


In [ ]:
!pip install astrapy

In [ ]:
from langchain.chains import ConversationalRetrievalChain
retriver = vector_store.as_retriever()
chain = ConversationalRetrievalChain.from_llm(
    llm = llm,
    retriever = retriver,

)

In [ ]:
chat_history = []

question1 = "What is the main idea of the Transformer architecture proposed in this paper?"
result1 = chain({"question": question1, "chat_history": chat_history})
print(f"🤔 Question: {question1}")
print(f"💬 Answer: {result1['answer']}")
print("-" * 80)

# Update chat history
chat_history.append((question1, result1['answer']))

# Example Question 2
question2 = "How does Multi-Head Attention work?"
result2 = chain({"question": question2, "chat_history": chat_history})
print(f"🤔 Question: {question2}")
print(f"💬 Answer: {result2['answer']}")
print("-" * 80)

# Example Question 3
question3 = "Why did the authors replace RNNs and convolutions?"
result3 = chain({"question": question3, "chat_history": chat_history})
print(f"🤔 Question: {question3}")
print(f"💬 Answer: {result3['answer']}")
print("-" * 80)



/tmp/ipython-input-3625892116.py:4: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result1 = chain({"question": question1, "chat_history": chat_history})
Token indices sequence length is longer than the specified maximum sequence length for this model (786 > 512). Running this sequence through the model will result in indexing errors


🤔 Question: What is the main idea of the Transformer architecture proposed in this paper?
💬 Answer: Stack self-attention and point-wise, fully connected layers for both the encoder and decoder
--------------------------------------------------------------------------------
🤔 Question: How does Multi-Head Attention work?
💬 Answer: Multi-head attention allows the model to jointly attend to information from different representation subspaces at different positions. With a single attention head, averaging inhibits this. MultiHead(Q, K, V) = Concat(head1, ...,headh)WO where headi = Attention(QWQ i , KWK i , V WV i ) Where the projections are parameter matricesWQ i  Rdmodeldk , WK i  Rdmodeldk , WV i  Rdmodeldv and WO  Rhdvdmodel . In this work we employ h = 8 parallel attention layers, or heads. For each of these we use dk = dv = dmodel/h = 64. Due to the reduced dimension of each head, the total computational cost is similar to that of single-head attention with full dimensionality. MultiH